# Assistant-03<br/>Function calling - Trading Assistant

## Credits

- https://mer.vin/2023/11/openai-assistants-api-function-calling/

### Get an OpenAI client and an Assistant

**Note:** When configuring the OpenAI client, the API version, the model version, and endpoint are "new".

In [1]:
import oaihelper as framework
import yfinance as yf

## Get an OpenAI client
client = framework.get_openai_client(api_key=framework.api_KEY,
        api_version=framework.api_version,
        azure_endpoint=framework.api_URI,)

### Retreive the latest stock price using Yahoo Finance

In [2]:
def get_stock_price(symbol: str) -> float:
    stock = yf.Ticker(symbol)
    price = stock.history(period="1d")['Close'].iloc[-1]
    return price

### Prepare the tool for function calling

In [3]:
tools_list = [{
    "type": "function",
    "function": {

        "name": "get_stock_price",
        "description": "Retrieve the latest closing price of a stock using its ticker symbol",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The ticker symbol of the stock"
                }
            },
            "required": ["symbol"]
        }
    }
}]

### Create the assistant

In [4]:
framework.clear_shelves()

## Create an assistant
stock_assistant = framework.create_assistant(client,
                                                  name="fta-Securities Trading Assistant",
                                                  instructions="You are a personal securities trading assistant. Please be polite, professional, helpful, and friendly.", 
                                                  tools=tools_list, 
                                                  model=framework.gpt_deployment_name)

Added assistant:  asst_KJ0leGJW4NZTDIYwogl3IqMK 1


### Handle Processing functions

In [5]:
def function_calling_delegate(client, thread, run):
    print("Function Calling")
    required_actions = run.required_action.submit_tool_outputs.model_dump()
    print(required_actions)
    tool_outputs = []
    import json
    for action in required_actions["tool_calls"]:
        func_name = action['function']['name']
        arguments = json.loads(action['function']['arguments'])
        
        if func_name == "get_stock_price":
            output = get_stock_price(symbol=arguments['symbol'])
            tool_outputs.append({
                "tool_call_id": action['id'],
                "output": output
            })
        else:
            raise ValueError(f"Unknown function: {func_name}")
        
    print("Submitting outputs back to the Assistant...")
    client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread.id,
        run_id=run.id,
        tool_outputs=tool_outputs
    )

### Process the messages, function calling leveraging the latest stock prices, print the results and dispose of the objects

In [6]:
framework.generate_response(client,stock_assistant,"Can you please provide me the latest stock prices for Microsoft and NVIDIA?", "123", "Alex", function_calling_delegate)
framework.generate_response(client,stock_assistant,"Can you please provide me the latest stock prices for Apple and Tesla?", "123", "Alex", function_calling_delegate)


Creating new thread for Alex with user_id 123
Added thread:  thread_EOZUA5kiRSWqB5ChzjTGp2gV 1
Function Calling
{'tool_calls': [{'id': 'call_hn9czgizwH5FlUwQdSPkZF4c', 'function': {'arguments': '{"symbol": "MSFT"}', 'name': 'get_stock_price'}, 'type': 'function'}, {'id': 'call_xJmC530Ap7xMdTYETx214z9y', 'function': {'arguments': '{"symbol": "NVDA"}', 'name': 'get_stock_price'}, 'type': 'function'}]}
Submitting outputs back to the Assistant...
Generated message: The latest closing prices for the stocks are as follows:

- Microsoft (MSFT): $388.76
- NVIDIA (NVDA): $561.76

Please note that stock prices fluctuate frequently during trading hours. These prices represent the last closing prices.
To Alex: The latest closing prices for the stocks are as follows:

- Microsoft (MSFT): $388.76
- NVIDIA (NVDA): $561.76

Please note that stock prices fluctuate frequently during trading hours. These prices represent the last closing prices.
Retrieving existing thread for Alex with user_id 123
Functi

'The latest closing prices for the stocks are as follows:\n\n- Apple (AAPL): $182.33\n- Tesla (TSLA): $220.13\n\nAs with all stock prices, these are subject to change with market conditions and the time of day. These figures represent the last closing prices.'

In [7]:
assistants, threads, files = framework.get_counts()
print(f"Framework counts Assistants: {assistants}, Threads: {threads}, Files: {files}")

Framework counts Assistants: 1, Threads: 1, Files: 0


In [8]:
framework.cleanup(client)

Deleting:  1  assistants.
AssistantDeleted(id='asst_KJ0leGJW4NZTDIYwogl3IqMK', deleted=True, object='assistant.deleted')
Deleting:  1  threads.
ThreadDeleted(id='thread_EOZUA5kiRSWqB5ChzjTGp2gV', deleted=True, object='thread.deleted')
Deleting:  0  files.
